In [76]:
# Convolutional neural network
# Made up of neurons
# Applies convolutional filters

In [77]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [78]:
#Device, gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#Hyper params
num_epochs = 2
batch_size = 1
learning_rate = 0.001

cuda


In [79]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_datasets = torchvision.datasets.FashionMNIST(root='./data', train=True,transform=transforms.ToTensor(), download=True)
test_datasets = torchvision.datasets.FashionMNIST(root='./data', train=False,transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_datasets, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_datasets, batch_size=batch_size, shuffle=False)

classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot')

In [80]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        #Colors: 3, output: 6, kernel_size: 5
        self.conv1 = nn.Conv2d(1,6,5)
        #kernel_size: 2, stride: 2
        self.pool = nn.MaxPool2d(2)
        #Input = conv1 output som er 6
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(64*2*2, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64*2*2)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = ConvNet().to(device)

In [81]:
#Loss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [82]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        #forward pass
        outputs = model(images)
        loss = criterion(outputs,labels)

        #backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if(i+1) % 10000 == 0:
            print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')

print('Finished Training')

epoch 1 / 2, step 2000/60000, loss = 2.3506
epoch 1 / 2, step 4000/60000, loss = 2.3470


KeyboardInterrupt: 

In [ ]:
#test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs,1)
        n_samples += labels.shape[0]
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if(label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

acc = 100.0 *n_correct/n_samples
print(f'Accuracy = {acc}')

for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]}: {acc} %')


Accuracy = 55.62
Accuracy of plane: 52.9 %
Accuracy of car: 62.9 %
Accuracy of bird: 45.6 %
Accuracy of cat: 32.4 %
Accuracy of deer: 57.5 %
Accuracy of dog: 45.9 %
Accuracy of frog: 57.8 %
Accuracy of horse: 66.2 %
Accuracy of ship: 80.4 %
Accuracy of truck: 54.6 %
